#### "PROMPT" Mask Example

In this example, we use a `maskPrompt` of _"sofa with pillows"_ to replace the sofa in the source image with a new sofa from the reference image.

_⚠️ **Warning:** Masks created with the “PROMPT” mask type will adhere tightly to the shape of the item you describe. This can be problematic in many scenarios because the product you are adding may not share the same silhouette or size as the item you are replacing. For this reason, the VTO API also provides an optional `maskShape` parameter that can be set to `“BOUNDING_BOX”`. We recommend using this setting (which is the default) in most cases when using the “PROMPT” mask type._

<div style="display: flex; gap: 2em; margin-right: 2em">
  <div style="text-align: center; flex: 1">
    <img src="../images/vto-images/vto_prompt_mask_source.jpg">
    <strong><tt>sourceImage</tt></strong>
  </div>
  <div style="text-align: center; flex: 1">
    <img src="../images/vto-images/vto_prompt_mask_reference.jpg">
    <strong><tt>referenceImage</tt></strong>
  </div>
   <div style="text-align: center; flex: 1">
    <img src="../images/vto-images/vto_prompt_mask_output.png">
    <strong>Example output</strong>
  </div>
</div>


In [ ]:
import logging
from datetime import datetime
from random import randint
import base64
from amazon_image_gen import BedrockImageGenerator
import file_utils

logging.basicConfig(format="[%(levelname)s] %(message)s", level=logging.INFO)

# Edit these values to experiment with your own images.
source_image_path = "../images/vto-images/vto_prompt_mask_source.jpg"
reference_image_path = "../images/vto-images/vto_prompt_mask_reference.jpg"

# Load the source image from disk.
with open(source_image_path, "rb") as image_file:
    source_image_base64 = base64.b64encode(image_file.read()).decode("utf-8")
# Load the reference image from disk.
with open(reference_image_path, "rb") as image_file:
    reference_image_base64 = base64.b64encode(image_file.read()).decode("utf-8")

inference_params = {
    "taskType": "VIRTUAL_TRY_ON",
    "virtualTryOnParams": {
        "sourceImage": source_image_base64,
        "referenceImage": reference_image_base64,
        "maskType": "PROMPT",
        "promptBasedMask": {
            "maskPrompt": "sofa with pillows",
            "maskShape": "BOUNDING_BOX",
        },
    },
    # The following is optional but provided here for you to experiment with.
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "quality": "standard",
        "cfgScale": 4.5,
        "seed": randint(0, 2147483646),
    },
}


# Define an output directory with a unique name.
generation_id = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output_directory = f"output/{generation_id}"

# Create the generator.
generator = BedrockImageGenerator(output_directory=output_directory)

# Generate the image(s).
response = generator.generate_images(inference_params)

if "images" in response:
    # Save and display each image
    images = file_utils.save_base64_images(
        response["images"], output_directory, "image"
    )
    for image in images:
        display(image)